# Pipeline de Análise de Risco de Mercado

## 1. Importações e Coleta de Dados

In [16]:
import yfinance as yf
import pandas as pd

In [17]:
# Definindo os tickers e o período
tickers = ['PETR4.SA', '^BVSP']
end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(months=6)

# Baixando os dados de fechamento, que já são ajustados por padrão
precos = yf.download(tickers, start=start_date, end=end_date)['Close']
precos = precos.dropna() # Garante que não há dias sem negociação para ambos os ativos

print("Amostra dos preços dos ativos:")
precos.head()

/tmp/ipykernel_248950/422196918.py:7: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  2 of 2 completed

Amostra dos preços dos ativos:


Ticker,PETR4.SA,^BVSP
Date,,
2025-02-03,35.524509,125970.0
2025-02-04,35.174004,125147.0
2025-02-05,34.918228,125534.0
2025-02-06,34.851913,126225.0
2025-02-07,34.652977,124619.0


## 2. Cálculo dos Indicadores de Risco

*(Esta seção será desenvolvida no próximo commit)*

## 3. Apresentação e Visualização dos Resultados

*(Esta seção será desenvolvida no próximo commit)*